In [237]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import hvplot.pandas

from matplotlib import pyplot as plt
import altair as alt
import numpy as np
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import esri2gpd
import carto2gpd
import seaborn as sns
import geopandas as gpd
import cenpy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from geopandas import GeoDataFrame
np.random.seed(42)

In [199]:
NPS = gpd.read_file("C:/Users/ssmat/Documents/NPS_-_National_Register_of_Historic_Places_Locations-shp/NPS_-_National_Register_of_Historic_Places_Locations.shp")

In [200]:
NPS = NPS[NPS['Status']!='Removed']
NPS = NPS[NPS['geometry'].isna() != True]

In [201]:
grped = NPS.groupby('State').count().reset_index()

In [202]:
# Add census data
acs = cenpy.remote.APIConnection("ACSDT5Y2018")

variables = ["NAME", "B01003_001E","B02008_001E","B23025_005E","B23025_001E","B29002_001E","B29002_006E","B29002_007E",
             "B29002_008E","B29003_001E","B29003_002E","B19001A_001E","B19001A_016E","B19001A_017E"]

In [203]:
state_data = acs.query(
    cols=variables,
    geo_unit="state:*",
)

In [66]:
state_data.head()

,NAME,B01003_001E,B02008_001E,B23025_005E,B23025_001E,B29002_001E,B29002_006E,B29002_007E,B29002_008E,B29003_001E,B29003_002E,B19001A_001E,B19001A_016E,B19001A_017E,B08126_001E,B08126_003E,state
0,Minnesota,5527358,4749380,119218,4379008,4037295,449792,906921,427276,3940147,344323,1907876,143913,143086,2879806,169795,27
1,Mississippi,2988762,1786914,110036,2350291,2228659,206747,279446,161062,2145906,384612,677533,30865,26200,1215697,79742,28
2,Missouri,6090062,5151022,155958,4864065,4585994,351725,764086,434106,4437928,546197,2017917,101409,94795,2855663,177468,29
3,Montana,1041732,953454,22433,838808,804263,67717,154949,73265,784859,100735,390495,17329,16291,499849,39939,30
4,Nebraska,1904760,1709927,36545,1483161,1358786,138686,272444,125607,1315343,128655,681092,36510,32127,980914,66644,31


In [204]:
for variable in variables:
    
    # Convert all variables EXCEPT for NAME
    if variable != "NAME":
        state_data[variable] = state_data[variable].astype(float)

In [205]:
# Convert to the data to floats
for col in variables[1:]:
    state_data[col] = state_data[col].astype(int)

state_data.columns = ['NAME', 'POPULATION','WHITE','UNEMPLOYED','TOTAL_EMP',"edu_total","edu_1","edu_2","edu_3",
                       "pov_tot","pov_below","inc_tot","inc_high1","inc_high2",'state']

In [206]:
state_data['white_per'] = state_data['WHITE'] / state_data['POPULATION'] *100

state_data['unemployed_per'] = state_data['UNEMPLOYED'] / state_data['TOTAL_EMP'] *100

state_data['edu_WithDegree_per'] = (state_data['edu_1'] + state_data['edu_2'] + state_data['edu_3']) / state_data['edu_total'] *100

state_data['BelowPoverty_per'] = state_data['pov_below'] / state_data['pov_tot'] *100

state_data['Above150_inc_per'] = (state_data['inc_high1'] + state_data['inc_high2']) / state_data['inc_tot'] *100

state_data = state_data[['NAME', 'POPULATION','white_per','unemployed_per','edu_WithDegree_per',
                           'BelowPoverty_per','Above150_inc_per']]

In [207]:
state_data['NAME'] = state_data['NAME'].str.upper() 

In [208]:
NPS_census = state_data.merge(
    grped,
    left_on=["NAME"],
    right_on=["State"],
)

In [209]:
NPS_census['LISTINGS_PER_POP'] = NPS_census['OBJECTID'] / NPS_census['POPULATION']

In [210]:
NPS_census = NPS_census[['NAME','OBJECTID', 'LISTINGS_PER_POP','POPULATION','white_per','unemployed_per','edu_WithDegree_per',
                           'BelowPoverty_per','Above150_inc_per']]

NPS_census.columns = ['NAME', 'NPS_count', 'LISTINGS_PER_POP','POPULATION','white_per','unemployed_per','edu_WithDegree_per',
                           'BelowPoverty_per','Above150_inc_per']

In [77]:
NPS_census.head()

,NAME,NPS_count,LISTINGS_PER_POP,POPULATION,white_per,unemployed_per,edu_WithDegree_per,BelowPoverty_per,Above150_inc_per
0,MINNESOTA,1370,0.000248,5527358,85.924957,2.722489,44.187730,8.738836,15.042854
1,MISSISSIPPI,888,0.000297,2988762,59.787765,4.681803,29.042352,17.923059,8.422468
2,MISSOURI,1672,0.000275,6090062,84.580781,3.206331,33.796752,12.307478,9.723096
3,MONTANA,894,0.000858,1041732,91.525843,2.674390,36.795302,12.834789,8.609585
4,NEBRASKA,789,0.000414,1904760,89.771257,2.463994,39.501217,9.781099,10.077493


In [186]:
ny_stats =  NPS_census.loc[NPS_census['NAME']=='NEW YORK']
ny_stats.head()

,NAME,NPS_count,LISTINGS_PER_POP,POPULATION,white_per,unemployed_per,edu_WithDegree_per,BelowPoverty_per,Above150_inc_per
9,NEW YORK,4481,0.000228,19618453,66.13496,3.777809,43.542552,11.995018,19.973243


In [211]:
acs.set_mapservice("tigerWMS_ACS2018")

Connection to American Community Survey: 1-Year Estimates: Detailed Tables 5-Year(ID: https://api.census.gov/data/id/ACSDT5Y2018)
With MapServer: Census Current (2018) WMS

In [212]:
# Query for states
all_states = acs.mapservice.layers[84].query(where="1=1")

C:\Users\ssmat\miniconda3\envs\musa-550-fall-2020\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [213]:
all_states = all_states[['NAME', 'geometry']]
all_states['NAME'] = all_states['NAME'].str.upper() 
all_states.head()

,NAME,geometry
0,DELAWARE,"POLYGON ((-8433572.860 4764277.969, -8433575.8..."
1,NEW MEXICO,"POLYGON ((-11800569.118 4438447.089, -11800457..."
2,CALIFORNIA,"MULTIPOLYGON (((-13818818.759 5081091.495, -13..."
3,NEW JERSEY,"POLYGON ((-8370067.985 4952313.911, -8370086.4..."
4,WISCONSIN,"POLYGON ((-10340141.000 5723505.920, -10340101..."


In [219]:
# Split the data 70/30
train_set, test_set = train_test_split(NPS_census, 
                                       test_size=0.3, 
                                       random_state=42)


# the target labels: log of sale price
y_train = np.log(train_set["NPS_count"])
y_test = np.log(test_set["NPS_count"])

# The features
num_cols = [
    "POPULATION",
    "white_per",
    "unemployed_per",
    "edu_WithDegree_per",
    "BelowPoverty_per",
    "Above150_inc_per",
]

X_train = train_set[num_cols].values
X_test = test_set[num_cols].values

In [220]:
# Make a linear model pipeline
linear_pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Fit on the training data
linear_pipeline.fit(X_train, y_train)

# What's the test score?
linear_pipeline.score(X_test, y_test)

0.13135479696033459

In [221]:
# Make a random forest pipeline
forest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
scores = cross_val_score(
    forest_pipeline,
    X_train,
    y_train,
    cv=10,
)

# Report
print("R^2 scores = ", scores)
print("Scores mean = ", scores.mean())
print("Score std dev = ", scores.std())

R^2 scores =  [ 0.68580286  0.3889235   0.42349024  0.48111395 -2.27683836  0.71298146
 -4.84152679 -0.03699293  0.45952657 -0.38348345]
Scores mean =  -0.43870029567125834
Score std dev =  1.6911573559545223


In [222]:
# Fit on the training data
forest_pipeline.fit(X_train, y_train)

# What's the test score? R-squared
forest_pipeline.score(X_test, y_test)

0.31285025774522934

In [223]:
# One-hot encoding in scikit learn

from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=100, 
                                       random_state=42)
)

# Fit the training set
pipe.fit(train_set, y_train);

# What's the test score?
pipe.score(test_set, y_test)

##it is the same because we don't have categorical data. Just making sure we are not missing a step.

0.31285025774522934

In [224]:
# making predictions:

model_name = "randomforestregressor"
param_grid = {
    f"{model_name}__n_estimators": [5, 10, 15, 20, 30],
    f"{model_name}__max_depth": [None, 2, 5, 7, 9, 13],
}

# Create the grid and use 5-fold CV
grid = GridSearchCV(pipe, param_grid, cv=5)

# Run the search
grid.fit(train_set, y_train);

grid.best_params_

{'randomforestregressor__max_depth': 2,
 'randomforestregressor__n_estimators': 5}

In [225]:
X = NPS_census.loc[test_set.index]

In [226]:
# Predictions for log of total trip counts
log_predictions = grid.best_estimator_.predict(test_set)

# Convert the predicted test values from log
X['prediction'] = np.exp(log_predictions)

In [227]:
# now I want to test the % error:

def evaluate(model, X_test, y_test):
    """
    Given a model and test features/targets, print out the 
    mean absolute error and accuracy
    """
    # Make the predictions
    predictions = model.predict(X_test)

    # Absolute error
    errors = abs(predictions - y_test)
    avg_error = np.mean(errors)

    # Mean absolute percentage error
    mape = 100 * np.mean(errors / y_test)

    # Accuracy
    accuracy = 100 - mape

    print("Model Performance")
    print(f"Average Absolute Error: {avg_error:0.4f}")
    print(f"Accuracy = {accuracy:0.2f}%.")

    return accuracy

In [230]:
# Evaluate on test set
evaluate(grid.best_estimator_, test_set, y_test)

Model Performance
Average Absolute Error: 0.5129
Accuracy = 92.23%.


92.23020743233143

In [231]:
# to create a column with % error

X["abs_error"] = (X["NPS_count"] - X["prediction"]).abs()

In [232]:
# Evaluate on test set
evaluate(grid.best_estimator_, test_set, y_test)

Model Performance
Average Absolute Error: 0.5129
Accuracy = 92.23%.


92.23020743233143

In [233]:
X["per_error"] = X["abs_error"]/X["NPS_count"]*100

In [234]:
X = all_states.merge(X, left_on='NAME', right_on='NAME')
X.head()

,NAME,geometry,NPS_count,LISTINGS_PER_POP,POPULATION,white_per,unemployed_per,edu_WithDegree_per,BelowPoverty_per,Above150_inc_per,prediction,abs_error,per_error
0,NEW MEXICO,"POLYGON ((-11800569.118 4438447.089, -11800457...",621,0.000297,2092434,77.117749,4.129199,33.525436,16.637527,8.847192,767.837349,146.837349,23.645306
1,WISCONSIN,"POLYGON ((-10340141.000 5723505.920, -10340101...",1788,0.000309,5778394,87.750057,2.689803,37.567537,10.398669,10.132417,1309.687152,478.312848,26.751278
2,NEBRASKA,"POLYGON ((-11583087.346 5161245.484, -11583086...",789,0.000414,1904760,89.771257,2.463994,39.501217,9.781099,10.077493,1063.026655,274.026655,34.730881
3,LOUISIANA,"POLYGON ((-10249066.406 3896364.550, -10249007...",1129,0.000242,4663616,63.935860,4.074247,27.398142,16.597788,11.793275,1143.617807,14.617807,1.294757
4,OHIO,"POLYGON ((-9440254.719 5010777.407, -9440253.7...",3150,0.000271,11641879,84.072176,3.677700,33.432183,12.450188,10.216980,1451.830334,1698.169666,53.910148


In [235]:
X_4326 = X.to_crs(epsg=4326)

In [238]:
states_4326 = all_states.to_crs(epsg=4326)

states_plot = alt.Chart(states_4326).mark_geoshape(stroke="grey",fill=None).encode().properties(
    width=325,
    height=200
).project('albersUsa')

### Which variables were most important?

In [166]:
# Create the data frame of importances
importance = pd.DataFrame(
    {"Feature": feature_cols, "Importance": forest_model.feature_importances_}
).sort_values("Importance")


importance.hvplot.barh(x="Feature", y="Importance")

:Bars   [Feature]   (Importance)

In [167]:
importance = importance.sort_values("Importance", ascending=False).iloc[:30]
importance.head(n=7)

,Feature,Importance
0,POPULATION,0.637854
3,edu_WithDegree_per,0.110886
2,unemployed_per,0.067768
1,white_per,0.067427
4,BelowPoverty_per,0.061467
5,Above150_inc_per,0.054598
